<a href="https://colab.research.google.com/github/anqi-guo/kaggle-nlp-with-disaster-tweets/blob/main/BERT_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

path = '/content/drive/My Drive/study/NLP_with_disaster_tweets/data'
df = pd.read_csv(f'{path}/train.csv')

df.drop_duplicates(subset=['text'], keep='first', inplace=True)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
import numpy as np
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):
  def __init__(self, df):
    self.labels = df['target'].tolist()
    self.texts = [tokenizer(text, padding='max_length', max_length=32, truncation=True, return_tensors='pt') for text in df['text']]

  def classes(self):
    return self.labels

  def __len__(self):
    return len(self.labels)

  def get_batch_labels(self, idx):
    return np.array(self.labels[idx])

  def get_batch_texts(self, idx):
    return self.texts[idx]

  def __getitem__(self, idx):
    batch_texts = self.get_batch_texts(idx)
    batch_y = self.get_batch_labels(idx)
    return batch_texts, batch_y

In [ ]:
np.random.seed(2022)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=2022), [int(.8*len(df)), int(.9*len(df))])

print(len(df_train), len(df_val), len(df_test))

6002 750 751


In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
  def __init__(self, dropout=.5):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, 2)
    self.relu = nn.ReLU()

  def forward(self, input_id, mask):
    # _ contains the embedding vectors of all the tokens in a sequence, pooled_output contains the embedding vector of [CLS] token
    _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False) 
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)
    final_layer = self.relu(linear_output)

    return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

  train ,val = Dataset(train_data), Dataset(val_data)

  train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
  val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

  device = torch.device('cuda')

  criterion = nn.CrossEntropyLoss()
  optimizer = Adam(model.parameters(), lr=learning_rate)

  model = model.cuda()
  criterion = criterion.cuda()

  for epoch_num in range(epochs):

    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader):
      
      train_label = train_label.to(device)
      mask = train_input['attention_mask'].to(device)
      input_id = train_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)

      batch_loss = criterion(output, train_label)
      total_loss_train += batch_loss.item()

      acc = (output.argmax(dim=1) == train_label).sum().item()
      total_acc_train += acc

      model.zero_grad()
      batch_loss.backward()
      optimizer.step()
    
    total_acc_val = 0
    total_loss_val = 0

    with torch.no_grad():

      for val_input, val_label in val_dataloader:

        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
    
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 3001/3001 [02:02<00:00, 24.58it/s]


Epochs: 1 | Train Loss:  0.302                 | Train Accuracy:  0.673                 | Val Loss:  0.234                 | Val Accuracy:  0.792


100%|██████████| 3001/3001 [02:01<00:00, 24.72it/s]


Epochs: 2 | Train Loss:  0.218                 | Train Accuracy:  0.819                 | Val Loss:  0.218                 | Val Accuracy:  0.815


100%|██████████| 3001/3001 [02:01<00:00, 24.75it/s]


Epochs: 3 | Train Loss:  0.182                 | Train Accuracy:  0.862                 | Val Loss:  0.203                 | Val Accuracy:  0.820


100%|██████████| 3001/3001 [02:01<00:00, 24.65it/s]


Epochs: 4 | Train Loss:  0.154                 | Train Accuracy:  0.885                 | Val Loss:  0.202                 | Val Accuracy:  0.839


100%|██████████| 3001/3001 [02:04<00:00, 24.06it/s]


Epochs: 5 | Train Loss:  0.128                 | Train Accuracy:  0.914                 | Val Loss:  0.217                 | Val Accuracy:  0.820


In [ ]:
path = '/content/drive/My Drive/study/NLP_with_disaster_tweets/data'
test_df = pd.read_csv(f'{path}/test.csv')

def predict(text):
  input = tokenizer(text, padding='max_length', max_length=32, truncation=True, return_tensors='pt')
  output = model(input['input_ids'].to(device), input['attention_mask'].to(device))
  label = output.argmax(dim=1).item()
  return label

from tqdm.auto import tqdm
tqdm.pandas()
test_df['target'] = test_df['text'].progress_apply(predict)

  0%|          | 0/3263 [00:00<?, ?it/s]

In [ ]:
test_df

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [ ]:
test_df[['id','target']].to_csv(f'{path}/submission.csv', index=False)